In [2]:
Spec_data = pd.DataFrame.from_dict(data = {"Vova":
            {"user_id": 259294833, "last_online" : 50, "age": -1, "in_work" : False},
              "Rodion":
            {"user_id": 11782046, "last_online" : 50, "age": -1, "in_work": False}}, orient = "index")

In [20]:
Spec_data[(Spec_data["in_work"] == False) & (Spec_data["user_id"] != 0) & 
          (Spec_data["last_online"] == Spec_data["last_online"].min())]['user_id']

Rodion     11782046
Vova      259294833
Name: user_id, dtype: int64

In [1]:
import logging
import pandas as pd
import re

from resend import USER_MESSAGE_FILTERS, resend_message 
from telegram import ReplyKeyboardMarkup, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler, CallbackQueryHandler,  PicklePersistence)


# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

(START, FIND_SPEC, ABOUT, FIND_SPEC_NOW, FIND_SPEC_LATER, SPEC_LIST, CHOOSE_AGE, PROBLEM_CATEGORY, 
 QUESTION_CATEGORY, CHOOSING_ABOUT, MESSAGES) = range(11)


def format_message(original_text):
    """
    Format original message text from user (adds header)
    :return: formatted original_text
    """
    return original_text


class Psy_Chat:
    def __init__(self, specialist_df_path = None):
        #self.Spec_data = pd.read_csv(specialist_df_path)
        self.Spec_data = pd.DataFrame.from_dict(data = {"Vova":
            {"user_id": 259294833, "last_online" : 50, "age": -1, "in_work" : False},
              "Rodion":
            {"user_id": 11782046, "last_online" : 50, "age": -1, "in_work": False}}, orient = "index")
        
    def start(self, update, context):
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec section.", user)
        # Build InlineKeyboard where each button has a displayed text
        # and a string as callback_data
        # The keyboard is a list of button rows, where each row is in turn
        # a list (hence `[[...]]`).
        keyboard = [
            [InlineKeyboardButton('Найти специалиста', callback_data=str(FIND_SPEC)),
             InlineKeyboardButton('О сервисе', callback_data=str(ABOUT))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        # Send message with text and appended InlineKeyboard
        update.effective_chat.send_message(
            text = """Привет!\n\nЭтот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят!\n\nДавай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
            reply_markup=reply_markup
    )
        return START

    
    def stop(self, update, context):
        """Закончить общение с ботом по команде /stop"""
        update.message.reply_text("""Пока!""")

        return ConversationHandler.END
    
    def done(self, update, context):
        update.message.reply_text("""Пока!""")
        user_data = context.user_data
        if 'choice' in user_data:
            del user_data['choice']

        print(user_data)

        user_data.clear()
        return ConversationHandler.END
    
    def find_spec(self, update, context):
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec section.", user)
        #Update message and keyboard
        query = update.callback_query
        bot = context.bot
        keyboard = [
            [InlineKeyboardButton('Быстрая помощь', callback_data=str(FIND_SPEC_NOW))],
            [InlineKeyboardButton('Подобрать специалиста', callback_data=str(FIND_SPEC_LATER))],
            [InlineKeyboardButton('Список специалистов', callback_data=str(SPEC_LIST))],
        #         [InlineKeyboardButton('Назад', callback_data=str(START))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        bot.edit_message_text(
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            text="Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.",
            reply_markup=reply_markup
            )

        return FIND_SPEC
    
    def find_spec_later(self, update, context):
    
        #IN PROGRESS

        # Section for complex search of specs
        user = update.effective_user.first_name
        logger.info("User %s goes to find_spec_later section.", user) 

        context.query = update.callback_query
        context.bot = context.bot

        context.user_data = pd.DataFrame(index = [update.effective_user.id], columns = ['gender', 'age', 'city', 'problem_category', 'statement_category'])
        for data in user_data.columns:
            if data == 'gender':
                def gender(update, context):
                    keyboard = [
                        [InlineKeyboardButton('Мужской', callback_data = "Мужской"),
                         InlineKeyboardButton('Женский', callback_data = "Женский")]
                    ]
                    reply_markup = InlineKeyboardMarkup(keyboard)
                    # Send message with text and appended InlineKeyboard
                    update.effective_chat.send_message(
                        text = "Твой пол?",
                        reply_markup=reply_markup
                    )
                    print(update.callback_query)
                    while update.inline_query == None:
                        print(update.callback_query)
                        user_data.loc[update.effective_user.id, 'gender'] = update.inline_query.query
                        print(user_data.loc[update.effective_user.id, 'gender'])
        pass
    
    def spec_list(self, update, context):
        pass
    
    def gender(self, update, context):
        r"""Пол"""
        reply_keyboard_loc = [['М', 'Ж']]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)

        update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
        return CHOOSE_AGE


    def age(self, update, context):
        r"""Возраст"""
        reply_keyboard_loc = [['< 16', '16-22', '22-40', "> 40"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""Выберите ваш возраст.""", reply_markup=markup_loc)

        return PROBLEM_CATEGORY

    def problem_category(self, update, context):
        r"""Выбрать проблему из предложенных типов"""
        reply_keyboard_loc = [['Семья', 'Друзья', 'Отношения', "Работа", "Учеба", "Другое"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""К какой категории Вы могли бы отнести свою психологическую проблему?""", reply_markup=markup_loc)

        return QUESTION_CATEGORY

    def statement_category(self, update, context):
        r"""Выбрать утверждение из предложенных типов"""
        reply_keyboard_loc = [['Я запутался/лась', 'Мне не интересно или скучно', 'Я устал', "Меня никто не слушает",
                               "Что делать дальше", "Как сделать правильный выбор", "Другое"]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""Какие утверждения или вопросы подходят Вам наиболее всего?""", reply_markup=markup_loc)

        return WAITING_TIME

    def find_spec_now(self, update, context):
        r"""Подбор специалиста сразу"""
        query = update.callback_query
        bot = context.bot
        keyboard = [
            [InlineKeyboardButton('Отменить поиск', callback_data=str(START))]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        bot.edit_message_text(
            chat_id=query.message.chat_id,
            message_id=query.message.message_id,
            text="Мы уже ищем специалиста. Ожидайте...",
            reply_markup=reply_markup
        )
        
        user_id = update._effective_user.id
        Best_specialist = self.Spec_data[(self.Spec_data["in_work"] == False) &
                                         (self.Spec_data["user_id"] != user_id)]['user_id']
        
        keyboard = [
            [InlineKeyboardButton('Принять клиента', callback_data = "Принять клиента"),
            InlineKeyboardButton('Отменить прием', callback_data = "Отменить прием")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        if len(Best_specialist) > 0:
            for paired_user_id in Best_specialist:       
                bot.sendMessage(paired_user_id, text= 'Приймите клиента если у вас есть возможность', reply_markup)
                if update.Message.ReplyToMessage.Text.Contains("Принять клиента"):
                    
                
            print(update)
        if user_id == 259294833:
            paired_user_id = 11782046
        else:
            paired_user_id = 259294833
            
        bot.sendMessage(user_id, text = 'Соединение установленно')
        bot.sendMessage(paired_user_id, text= 'Соединение установленно')
        
        return MESSAGES
    
    def message(self, update, context):

        user_id = update._effective_user.id
        bot = context.bot
        #self.Spec_data[self.Spec_data["user_id"] != user_id]
        if user_id == 259294833:
            paired_user_id = 11782046
        else:
            paired_user_id = 259294833
            
        if paired_user_id and paired_user_id > 0:
            logger.info('Resending ' + str(update.message.text))
            try:
                resend_message(bot, update, paired_user_id, format_message)
                print("resended")
            except Exception as E:
                print("fail "+ str(E))
                bot.sendMessage(user_id, text='DISCONNECTED',
                                #reply_markup=KEYBOARD_MARKUP
                               )
        else:
            print("FAIL")

    def about(self, update, context):
        r"""О нас"""
        reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
        markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
        update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)

        return CHOOSING_ABOUT

    def error(self, update, context):
        """Log Errors caused by Updates."""
        logger.warning('Update "%s" caused error "%s"', update, context.error)
        
    def run(self):
        my_persistence = PicklePersistence(filename='my_file')
        
        updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", persistence=my_persistence,
                          use_context=True)
        
        # Get the dispatcher to register handlers
        dp = updater.dispatcher

        conv_handler = ConversationHandler(
            entry_points=[CommandHandler('start', self.start)],
            states={
                START: [CallbackQueryHandler(self.find_spec, pattern='^' + str(FIND_SPEC) + '$'),
                       CallbackQueryHandler(self.about, pattern='^' + str(ABOUT) + '$')],

                FIND_SPEC: [CallbackQueryHandler(self.find_spec_now, pattern='^' + str(FIND_SPEC_NOW) + '$'),
                           CallbackQueryHandler(self.find_spec_later, pattern='^' + str(FIND_SPEC_LATER) + '$'),
                           CallbackQueryHandler(self.spec_list, pattern='^' + str(SPEC_LIST) + '$')],
                
                MESSAGES: [MessageHandler(filters = Filters.text | Filters.audio | Filters.document | Filters.photo | Filters.sticker | Filters.video | Filters.voice | Filters.contact | Filters.location | Filters.venue
                           ,callback= self.message)]

    #             FIND_SPEC_NOW: [CallbackQueryHandler(start, pattern='^' + str(START) + '$')],

    #             ABOUT: [CallbackQueryHandler(start_over, pattern='^' + str(ONE) + '$'),
    #                      CallbackQueryHandler(end, pattern='^' + str(TWO) + '$')]
            },
            fallbacks=[CommandHandler('start', self.start),
                      CallbackQueryHandler(self.start, pattern='^' + str(START) + '$')
                      ]
        )    

        dp.add_handler(conv_handler)
        print(dp.handlers)
        # log all errors
        dp.add_error_handler(self.error)

        # Start the Bot
        updater.start_polling()

        # Run the bot until you press Ctrl-C or the process receives SIGINT,
        # SIGTERM or SIGABRT. This should be used most of the time, since
        # start_polling() is non-blocking and will stop the bot gracefully.
        updater.idle()

In [2]:
a = Psy_Chat()

In [ ]:
a.run()

C:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\conversationhandler.py:215: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn("If 'per_message=False', 'CallbackQueryHandler' will not be "


{0: [<telegram.ext.conversationhandler.ConversationHandler object at 0x000001D33352ABC8>]}


2019-12-16 16:29:11,707 - __main__ - INFO - User Родион goes to find_spec section.
2019-12-16 16:29:18,859 - __main__ - INFO - User Родион goes to find_spec section.
2019-12-16 16:29:24,584 - __main__ - INFO - Resending {'message_id': 1177, 'date': 1576492177, 'chat': {'id': 11782046, 'type': 'private', 'username': 'Rpl123', 'first_name': 'Родион'}, 'text': '123123123123', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 11782046, 'first_name': 'Родион', 'is_bot': False, 'username': 'Rpl123', 'language_code': 'ru'}}


i_am_sending_text
123123123123
resended


In [21]:
d = {'update_id': 716208453, 'callback_query': {'id': '50603506090824630', 'chat_instance': '6628734935264516976', 'message': 
    {'message_id': 937, 'date': 1576484478, 'chat': {'id': 11782046, 'type': 'private', 'username': 'Rpl123', 'first_name': 'Родион'}
     , 'edit_date': 1576484480, 'text': 'Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'reply_markup': {'inline_keyboard': [[{'text': 'Быстрая помощь', 'callback_data': '3'}], [{'text': 'Подобрать специалиста', 'callback_data': '4'}], [{'text': 'Список специалистов', 'callback_data': '5'}]]}, 'from': {'id': 984001690, 'first_name': 'ПомощьПсихолога', 'is_bot': True, 'username': 'HelpPsychoRu_bot'}}, 'data': '3', 'from': {'id': 11782046, 'first_name': 'Родион', 'is_bot': False, 'username': 'Rpl123', 'language_code': 'ru'}}, '_effective_user': {'id': 11782046, 'first_name': 'Родион', 'is_bot': False, 'username': 'Rpl123', 'language_code': 'ru'}, '_effective_chat': {'id': 11782046, 'type': 'private', 'username': 'Rpl123', 'first_name': 'Родион'}}

In [25]:
d['callback_query']['message']

{'message_id': 937,
 'date': 1576484478,
 'chat': {'id': 11782046,
  'type': 'private',
  'username': 'Rpl123',
  'first_name': 'Родион'},
 'edit_date': 1576484480,
 'text': 'Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.',
 'entities': [],
 'caption_entities': [],
 'photo': [],
 'new_chat_members': [],
 'new_chat_photo': [],
 'delete_chat_photo': False,
 'group_chat_created': False,
 'supergroup_chat_created': False,
 'channel_chat_created': False,
 'reply_markup': {'inline_keyboard': [[{'text': 'Быстрая помощь',
     'callback_data': '3'}],
   [{'text': 'Подобрать специалиста', 'callback_data': '4'}],
   [{'text': 'Список специалистов', 'callback_data': '5'}]]},
 'from': {'id': 984001690,
  'first_name': 'ПомощьПсихолога',
  'is_bot': True,
  'username': 'HelpPsychoRu_bot'}}

In [23]:
d.keys()

dict_keys(['update_id', 'callback_query', '_effective_user', '_effective_chat'])

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.

"""
First, a few callback functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Example of a bot-user conversation using ConversationHandler.
Send /start to initiate the conversation.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [['Age', 'Favourite colour'],
                  ['Number of siblings', 'Something else...'],
                  ['Done']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)


reply_keyboard_2 = [""]
def facts_to_str(user_data):
    facts = list()

    for key, value in user_data.items():
        facts.append('{} - {}'.format(key, value))

    return "\n".join(facts).join(['\n', '\n'])


def start(update, context):
    update.message.reply_text(
        "Hi! My name is Doctor Botter. I will hold a more complex conversation with you. "
        "Why don't you tell me something about yourself?",
        reply_markup=markup)

    return CHOOSING


def regular_choice(update, context):
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text(
        'Your {}? Yes, I would love to hear about that!'.format(text.lower()))

    return TYPING_REPLY


def custom_choice(update, context):
    update.message.reply_text('Alright, please send me the category first, '
                              'for example "Most impressive skill"')

    return TYPING_CHOICE


def received_information(update, context):
    user_data = context.user_data
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']

    update.message.reply_text("Neat! Just so you know, this is what you already told me:"
                              "{} You can tell me more, or change your opinion"
                              " on something.".format(facts_to_str(user_data)),
                              reply_markup=markup)

    return CHOOSING


def done(update, context):
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    update.message.reply_text("I learned these facts about you:"
                              "{}"
                              "Until next time!".format(facts_to_str(user_data)))

    user_data.clear()
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^(Age|Favourite colour|Number of siblings)$'),
                                      regular_choice),
                       MessageHandler(Filters.regex('^Something else...$'),
                                      custom_choice)
                       ],

            TYPING_CHOICE: [MessageHandler(Filters.text,
                                           regular_choice)
                            ],

            TYPING_REPLY: [MessageHandler(Filters.text,
                                          received_information),
                           ],
        },

        fallbacks=[MessageHandler(Filters.regex('^Done$'), done)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

In [19]:
import logging

import re

from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, STEP_2_FIND_SPEC , CHOOSE_AGE, PROBLEM_CATEGORY, QUESTION_CATEGORY = range(5)
FIND_SPEC_NOW, CHOOSING_ABOUT = range(5,7)



def start(update, context):
    reply_keyboard = [['Найти специалиста', 'О сервисе']]

    markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
    update.message.reply_text(
        """Привет! Этот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят! 
        Давай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", 
        так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
        reply_markup=markup)

    return CHOOSING

def stop(update, context):
    """Закончить общение с ботом по команде /stop"""
    update.message.reply_text("""Пока!""")
    
    return ConversationHandler.END

def done(update, context):
    update.message.reply_text("""Пока!""")
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    print(user_data)

    user_data.clear()
    return ConversationHandler.END

def Find_Spec(update, context):
    r"""найи специалиста выбор из двух альтернатив и переход к след шагу"""
    reply_keyboard_loc = [['Быстрая помощь','Подобрать', 'Список Специалистов']]

    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
    update.message.reply_text(
        """Мы можем помочь подобрать Вам специалиста сразу или в зависимости от характера вашей проблемы, 
        либо вы можете выбрать его самостоятельно из списка специалистов.""",
        reply_markup=markup_loc)
    
    return STEP_2_FIND_SPEC

def Pol(update, context):
    r"""Пол"""
    reply_keyboard_loc = [['М', 'Ж']]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
    update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
    return CHOOSE_AGE


def Age(update, context):
    r"""Возраст"""
    reply_keyboard_loc = [['< 16', '16-22', '22-40', "> 40"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Выберите ваш возраст.""", reply_markup=markup_loc)
    
    return PROBLEM_CATEGORY

def Problem_Category(update, context):
    r"""Выбрать проблему из предложенных типов"""
    reply_keyboard_loc = [['Семья', 'Друзья', 'Отношения', "Работа", "Учеба", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""К какой категории Вы могли бы отнести свою психологическую проблему?""", reply_markup=markup_loc)
    
    return QUESTION_CATEGORY

def Question_Category(update, context):
    r"""Выбрать вопрос из предложенных типов"""
    reply_keyboard_loc = [['Я запутался/лась', 'Мне не интересно или скучно', 'Я устал', "Меня никто не слушает",
                           "Что делать дальше", "Как сделать правильный выбор", "Другое"]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""Какие утверждения или вопросы подходят Вам наиболее всего?""", reply_markup=markup_loc)
    
    return WAITING_TIME

def find_spec_now(update, contex):
    r"""Подбор специалиста сразу"""
    update.message.reply_text("""Мы уже ищем специалиста. Ожидайте...""")
    ''' Нужно допилить алгоритм быстрого поиска'''
    def fast_search(update, context):
        pass
    
    return FIND_SPEC_NOW

def about(update, context):
    r"""О нас"""
    reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)
    
    return CHOOSING_ABOUT
    
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^Найти специалиста$'),
                                      Find_Spec),
                       MessageHandler(Filters.regex('^О сервисе$'),
                                      about),
                       ],
            STEP_2_FIND_SPEC: [ MessageHandler(Filters.regex('^Быстрая помощь$'),
                                      find_spec_now),
                               MessageHandler(Filters.regex('^Подобрать$'),
                                      Pol),
                       MessageHandler(Filters.regex('^Список Специалистов$'),
                                      Pol)
            ],

            CHOOSE_AGE: [MessageHandler(Filters.regex('^.*$'), Age) 
                            ],
            
            PROBLEM_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Problem_Category)
                            ],
            
            QUESTION_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Question_Category)
                            ],
        },

        fallbacks=[MessageHandler(Filters.regex('^Done$'), done),
                  CommandHandler('stop', stop)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2019-12-05 22:52:32,858 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555C50A48>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-05 22:52:37,876 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEF88>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-05 22:52:42,877 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, 

2019-12-08 09:06:22,604 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEB48>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))"
2019-12-08 09:06:29,111 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555D98CC8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:06:34,123 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connec

2019-12-08 09:07:54,937 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA13C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2019-12-08 09:07:54,937 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA13C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))"
2019-12-08 09:08:07,546 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Ret

2019-12-08 09:10:11,771 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001655620F4C8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:10:16,783 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AA1248>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
2019-12-08 09:10:16,783 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPo

2019-12-08 09:13:35,399 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEEC8>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:13:40,411 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x0000016555AEEB08>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)')': /bot984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs/getUpdates
2019-12-08 09:13:45,423 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host